In [1]:
import csv
import itertools
import numpy as np
import numpy.typing as npt
import matplotlib.pyplot as plt
import torch

from regions_of_interest_module.regions_of_interest import find_points_of_interest
from image_manipulation.crop_image_for_classifier import crop, is_valid_size
from image_loader.Image2numpy import convert_image_file_to_numpy
from image_manipulation.standardize_image import normalize_image

# now comes the neural network
from noise_reduction.model_denoise import DenoiseNet
from spot_classifier_NN.classifier_model import spots_classifier_net as ClassifierNet

from image_loader.load_tagged_image import load_tagged_image
from create_training_data.training_data_generator import TaggedImagesGenerator

NUM_OF_ROI_IN_BATCH = 300

In [2]:
image_cleaner_network = DenoiseNet()
image_cleaner_network.load_state_dict(torch.load('denoiseNet.pt', map_location=torch.device('cpu')))

classifier_network = ClassifierNet()
classifier_network.load_state_dict(torch.load("classifyNet.pt", map_location=torch.device('cpu')))

<All keys matched successfully>

In [3]:
def clean_image(image: npt.NDArray) -> npt.NDArray:
    image = torch.tensor(image)  # type: ignore
    image = image.permute(3, 2, 1, 0)  # get ch,z,x,y # type: ignore
    image_real = image.unsqueeze(1)  # type: ignore
    pred = image_cleaner_network(image_real.float()).detach()
    pred = pred[:, 0, :, :, :]
    pred = pred.permute(3, 2, 1, 0)
    return pred.numpy()

def classify_roi_list(small_images_list: npt.NDArray, cutoff=0.95):
    images = torch.tensor(small_images_list)  # type: ignore
    images = images.permute(0, 4, 3, 2, 1)  # get ch,z,x,y # type: ignore
    pred = classifier_network(images.float()).detach()
    pred = torch.sigmoid(pred)
    has_point = pred.numpy() > cutoff
    result_channel = np.zeros(has_point.shape)
    return has_point, result_channel

In [ ]:
import os


for i in range(1,7):
    fig,axes = plt.subplots(2,3,figsize=(23,15))
    axes = iter(axes.flatten())
    
    ogimage, tags = load_tagged_image(f"images\\tagged_images_validation\\img{i}")

    image = normalize_image(ogimage)
    image = image[..., :3]
    image = clean_image(image)

    points_of_interest = list(zip(*np.where(find_points_of_interest(image))))  # type: ignore
    points_of_interest: list[tuple[int, int, int]]  # points_of_interest = list of (x,y,z) points

    # prepare a batch of small images
    small_images_arr = []
    small_images_center_location_arr = []
    for point in points_of_interest:
        small_image = crop(ogimage, point[0], point[1])
        if not is_valid_size(small_image):
            continue
        small_images_arr.append(small_image)
        small_images_center_location_arr.append(point)
    small_images_arr = np.array(small_images_arr)
    small_images_center_location_arr = np.array(small_images_center_location_arr)
    
    for cutoff in [0.5,0.7,0.85,0.9,0.95,0.985]:
        
        small_images_arr_copy = np.copy(small_images_arr)
        has_point_list, result_channels_list = classify_roi_list(small_images_arr_copy, cutoff)
        
        ch = int(tags[-1][3])
        ax = next(axes)
        ax.imshow(np.max(ogimage[...,ch],axis=-1),cmap="gray")
        ax.plot(tags[:,0],tags[:,1],".g",markersize=3)
        ax.plot(small_images_center_location_arr[has_point_list,1],small_images_center_location_arr[has_point_list,0],".r",markersize=1)
        ax.set_title(f"image:{i}, cutoff:{cutoff}, shown_channel:{ch}")
    fig.savefig(os.path.join("result_images",f"im{i}.tiff"))

In [ ]:
import os

image_gen = TaggedImagesGenerator.make_default_training_data_generator(batch_size=1)

for i in range(1,7):
    fig,axes = plt.subplots(2,3,figsize=(23,15))
    axes = iter(axes.flatten())
    ogimage, tags = image_gen.get_next_batch()[0]
    
    image = normalize_image(ogimage)
    image = image[..., :3]
    image = clean_image(image)

    points_of_interest = list(zip(*np.where(find_points_of_interest(image))))  # type: ignore
    points_of_interest: list[tuple[int, int, int]]  # points_of_interest = list of (x,y,z) points

    # prepare a batch of small images
    small_images_arr = []
    small_images_center_location_arr = []
    for point in points_of_interest:
        small_image = crop(ogimage, point[0], point[1])
        if not is_valid_size(small_image):
            continue
        small_images_arr.append(small_image)
        small_images_center_location_arr.append(point)
    small_images_arr = np.array(small_images_arr)
    small_images_center_location_arr = np.array(small_images_center_location_arr)
    
    for cutoff in [0.5,0.7,0.85,0.9,0.95,0.985]:
        
        small_images_arr_copy = np.copy(small_images_arr)
        has_point_list, result_channels_list = classify_roi_list(small_images_arr_copy, cutoff)
        
        ch = int(tags[-1][-1])
        ax = next(axes)
        ax.imshow(np.max(ogimage[...,ch],axis=-1),cmap="gray")
        ax.plot(tags[:,0],tags[:,1],".g",markersize=3)
        ax.plot(small_images_center_location_arr[has_point_list,1],small_images_center_location_arr[has_point_list,0],".r",markersize=1)
        ax.set_title(f"image:{i}, cutoff:{cutoff}, shown_channel:{ch}")
    fig.savefig(os.path.join("result_images",f"synth_im{i}.tiff"))

NameError: name 'TaggedImagesGenerator' is not defined